In [12]:
import numpy as np
import os
import keras.backend as K
from keras.layers import Embedding, Reshape, Merge, Dropout, Dense
from keras.models import Sequential

In [5]:
train_data_path = '/home/csdai/hw5_data/train.csv'
test_data_path = '/home/csdai/hw5_data/test.csv'
answer_path = '/home/csdai/hw5_data/answer.txt'

In [6]:
def read_train_data(train_data_path):
    with open(train_data_path, 'r') as train_data:
        train_data.readline()
        users = []
        movies = []
        ratings = []
        for data in train_data:
            lt = data.strip().split(',')
            users.append(int(lt[1]))
            movies.append(int(lt[2]))
            ratings.append(int(lt[3]))
    return users, movies, ratings

def read_test_data(test_data_path):
    with open(test_data_path, 'r') as test_data:
        test_data.readline()
        users = []
        movies = []
        for data in test_data:
            lt = data.strip().split(',')
            users.append(int(lt[1]))
            movies.append(int(lt[2]))
    return users, movies

users, movies, ratings = read_train_data(train_data_path)

In [7]:
def rmse(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

In [49]:
def mf_model(user_num, movie_num, factors):
    u = Sequential()
    u.add(Embedding(user_num, factors, input_length=1))
    u.add(Reshape((factors,)))
    m = Sequential()
    m.add(Embedding(movie_num, factors, input_length=1))
    m.add(Reshape((factors,)))
    model = Sequential()
    model.add(Merge([u, m], mode='dot'))
    model.compile(loss='mse', optimizer='adamax')
    return model

In [50]:
model = mf_model(max(users), max(movies), 256)
model.fit([np.array(users), np.array(movies)], ratings, batch_size=512, nb_epoch=1, verbose=1)

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  if __name__ == '__main__':
/usr/lib64/python3.6/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1
899873/899873 [==============================] - 27s 30us/step - loss: 9.2521


In [55]:
model.fit([np.array(users), np.array(movies)], ratings, batch_size=512, nb_epoch=3, verbose=1)

/usr/lib64/python3.6/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/3
899873/899873 [==============================] - 27s 30us/step - loss: 0.6877
Epoch 2/3
899873/899873 [==============================] - 27s 30us/step - loss: 0.6562
Epoch 3/3
899873/899873 [==============================] - 27s 30us/step - loss: 0.6221


In [56]:
model.fit([np.array(users), np.array(movies)], ratings, batch_size=256, nb_epoch=1, verbose=1)

/usr/lib64/python3.6/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1
899873/899873 [==============================] - 53s 59us/step - loss: 0.5889


In [36]:
test_users, test_movies = read_test_data(test_data_path)

In [57]:
def predict_rating(userid, movieid):
    return model.predict([np.array([userid]), np.array([movieid])])[0][0]

def predict(answer_path, model, test_data_path):
    if os.path.isfile(answer_path):
        os.remove(answer_path)
    test_users, test_movies = read_test_data(test_data_path)
    pred = []
    for i in range(len(test_users)):
        pred.append(predict_rating(test_users[i], test_movies[i]))
    return pred
    
pred = predict(answer_path, model, test_data_path)

In [58]:
print(pred)

[4.1740756, 4.0872355, 4.0767283, 2.8327768, 3.3130674, 3.7317204, 4.5939713, 3.0984218, 3.109288, 2.7867532, 4.1273189, 3.5556164, 3.4818859, 3.7770228, 3.176125, 3.5364656, 3.7411265, 3.8150158, 3.9534068, 3.4987242, 4.1388235, 4.4262276, 3.7790749, 3.2283826, 3.5248752, 2.9901657, 3.6682627, 3.6566551, 2.1535776, 3.2223735, 2.9906259, 3.3629861, 3.656219, 3.0359206, 4.3658695, 4.0652294, 3.905067, 3.0355506, 3.3094258, 2.0377681, 3.6224492, 3.777842, 2.7473216, 4.5616441, 3.9124444, 3.6588225, 3.467205, 3.8504715, 3.4976163, 3.6017799, 3.9115038, 3.2400126, 4.222014, 4.6790428, 4.131062, 3.3478036, 4.3304234, 2.7961576, 3.916379, 3.5939875, 3.4390626, 4.2648978, 4.1983004, 4.2117724, 4.3143821, 4.0929055, 4.0367761, 4.2726703, 3.1675241, 4.5154386, 3.2281733, 4.5029988, 3.3221371, 4.1947074, 2.9934359, 2.9044495, 4.1531019, 4.3433461, 4.3243766, 4.2770033, 4.239924, 3.7578974, 4.5997334, 4.6971145, 4.8372126, 3.8946233, 4.7172484, 4.2629995, 4.1609674, 4.4990597, 4.2554088, 3.931678

In [59]:
with open(answer_path, 'a') as answer:
    answer.write('TestDataID,Rating\r\n')
    index = 1
    for ele in pred:
        answer.write(str(index) +','+ str(ele) +'\r\n')
        index += 1